In [1]:
from GraphSummarizers.Coarsener.HeteroCoarsener import HeteroCoarsener
from Datasets.NodeClassification.DBLP import DBLP
from Datasets.NodeClassification.AIFB import AIFB
from Datasets.NodeClassification.TestHetero import TestHeteroSmall, TestHeteroBig
import importlib
import torch
from test_data_converter import dgl_to_pyg_input

/home/robin/uu/thesis/convolution-matching/.venvconv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
aifb = AIFB()
new_g = aifb.load_graph()


Done loading data from cached files.


In [ ]:
new_g.nodes("Forschungsgebiete")

coarsener = HeteroCoarsener(None, new_g, 0.5)
H = coarsener._create_h_spatial_rgcn(new_g)
candidates = coarsener._select_candidates(coarsener._get_intersection(coarsener._get_rgcn_edges(H)))


#

start create H


/home/robin/uu/thesis/convolution-matching/GraphSummarizers/Coarsener/HeteroCoarsener.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  degree = torch.tensor(self.node_degrees[etype]["out"]).to(device)
/home/robin/uu/thesis/convolution-matching/GraphSummarizers/Coarsener/HeteroCoarsener.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  degree = torch.tensor(self.node_degrees[etype]["in"]).to(device)


created H 23.247557640075684
start init costs
stop init costs 3.739961624145508
start lowest cost edges
stop lowest cost edges 2.5494067668914795
start select candidates
stop select canidates 0.03641319274902344


In [4]:
merged_graph, mapping_authors = coarsener._merge_nodes(new_g, "Forschungsgebiete", candidates["Forschungsgebiete"])

In [5]:
original_data, o_x_dict, o_edge_index_dict, o_node_types, o_edge_types = dgl_to_pyg_input(new_g)

coarsened_data, c_x_dict, c_edge_index_dict, c_node_types, c_edge_types = dgl_to_pyg_input(merged_graph)

/home/robin/uu/thesis/convolution-matching/.venvconv/lib/python3.10/site-packages/torch_geometric/data/hetero_data.py:573: UserWarning: There exist type names in the 'HeteroData' object that contain double underscores '__' (e.g., 'Forschungsgebiete_ontology#name__Literal'). This may lead to unexpected behavior. To avoid any issues, ensure that your type names only contain single underscores.
  warnings.warn(f"There exist type names in the "


In [6]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data.rdf import AIFBDataset
from Models.GNNs.HGCN import RGCN

from sklearn.metrics import accuracy_score

# Load AIFB dataset
dataset = AIFBDataset()
g = dataset[0]

# Check node types and label node type
print("Node types:", g.ntypes)
print("Label node type:", dataset.predict_category)  # should be 'Personen'
#g = merged_graph
label_ntype = dataset.predict_category  # 'Personen'
labels = g.nodes[label_ntype].data['labels']
train_mask = g.nodes[label_ntype].data['train_mask'].bool()
num_classes = dataset.num_classes
test_mask = g.nodes[label_ntype].data['test_mask'].to(torch.bool)
# Define embedding size
embed_size = 16
#g = merged_graph
# Create trainable input embeddings for all node types
node_embeds = nn.ModuleDict({
    ntype: nn.Embedding(g.num_nodes(ntype), embed_size)
    for ntype in g.ntypes
})

# Instantiate model
model = RGCN(in_feats=embed_size, hidden_feats=32, out_feats=num_classes, etypes=g.etypes)

# Move to device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
node_embeds = node_embeds.to(device)
labels = labels.to(device)
train_mask = train_mask.to(device)
g = g.to(device)

# Optimizer
optimizer = torch.optim.Adam(list(model.parameters()) + list(node_embeds.parameters()), lr=0.01)

# Training loop
for epoch in range(50):
    model.train()
    
    inputs = {ntype: node_embeds[ntype].weight for ntype in g.ntypes}
    logits = model(g, inputs)[label_ntype]
    
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    pred = logits.argmax(dim=1)
    acc = accuracy_score(labels[train_mask].cpu(), pred[train_mask].cpu())
    if epoch % 10 == 0:
        # Print training loss and accuracy every 10 epochs
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | Train Acc: {acc:.4f}")

test_mask = g.nodes[label_ntype].data['test_mask'].to(torch.bool)

# Evaluate
model.eval()
with torch.no_grad():
    inputs = {ntype: node_embeds[ntype].weight for ntype in g.ntypes}
    logits = model(g, inputs)[label_ntype]
    pred = logits.argmax(dim=1)

    test_acc = accuracy_score(labels[test_mask].cpu(), pred[test_mask].cpu())
    print(f"\n✅ Test Accuracy on '{label_ntype}' nodes: {test_acc:.4f}")



Done loading data from cached files.
Node types: ['Forschungsgebiete', 'Forschungsgruppen', 'Kooperationen', 'Personen', 'Projekte', 'Publikationen', '_Literal']
Label node type: Personen
Epoch 000 | Loss: 5.6058 | Train Acc: 0.1643
Epoch 010 | Loss: 0.0926 | Train Acc: 0.9786
Epoch 020 | Loss: 0.0351 | Train Acc: 0.9929
Epoch 030 | Loss: 0.0114 | Train Acc: 1.0000
Epoch 040 | Loss: 0.0046 | Train Acc: 1.0000

✅ Test Accuracy on 'Personen' nodes: 0.8889


In [7]:
test_mask = g.nodes[label_ntype].data['test_mask'].to(torch.bool)

# Evaluate
model.eval()
with torch.no_grad():
    inputs = {ntype: node_embeds[ntype].weight for ntype in g.ntypes}
    logits = model(g, inputs)[label_ntype]
    pred = logits.argmax(dim=1)

    test_acc = accuracy_score(labels[test_mask].cpu(), pred[test_mask].cpu())
    print(f"\n✅ Test Accuracy on '{label_ntype}' nodes: {test_acc:.4f}")



✅ Test Accuracy on 'Personen' nodes: 0.8889
